# Assignment 5 - Population

For the PFDA Module

Author: Kyra Menai Hamilton

# DELETE LATER
## ASSIGNMENT REQUIREMENTS

Part 1 70%
Write a jupyter notebook that analyses the differences between the sexes by age in Ireland.

Weighted mean age (by sex)
The difference between the sexes by age
This part does not need to look at the regions.

ie You can take the notebook I used in the lectures and substitute the sexes for the regions.

Part 2 20%
In the same notebook, make a variable that stores an age (say 35).

Write that code that would group the people within 5 years of that age together, into one age group 

Calculate the population difference between the sexes in that age group.

Part 3 10%
In the same notebook.

Write the code that would work out which region in Ireland has the biggest population difference between the sexes in that age group

## Import Packages

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Getting the data

## Cleaning the data


## Analysing the data

### Part 1 - Differenences between the sexes by age in 
- Weighted MEAN Age (by sex) - Regions not necessary 

### Part 2 - Make a variable that stores an age
- Write the codee that would group the people within 5 years of that age together into one age group
- Calculate the population difference between the sexes in that age group

### Part 3 - Regions vs sexes
- Write the code that would work out which region in Ireland has the biggest population difference between the sexes in that age group.